# Clasificación usando Python

## Ejemplo: Iris Dataset

Vamos a ocupar el dataset **iris** disponible en sklearn, que como hemos visto contiene 150 **instancias** (filas) de 3 **clases** diferentes de flores. El método **load_iris** permite cargar el dataset.

In [ ]:
#Para cargar las bibliotecas, haz click en el siguiente bloque de código, y ejecútalo presionando `Shift+Enter`:
from sklearn.datasets import load_iris
import pandas as pd
import numpy as np
print("Si se muestra esto es porque ya está instalado!")

In [ ]:
iris = load_iris()

In [ ]:
iris

In [ ]:
iris_df = pd.DataFrame(data= np.c_[iris['data'], iris['target']],
                     columns= iris['feature_names'] + ['target'])

In [ ]:
iris_df

In [ ]:
X = np.array(iris_df.iloc[: , :4])    ## datos, caracteristicas o features de cada flor. 

y = iris_df.target    ## clase para cada instancia anterior.

print("X:\n", X[:10])   # muestra las primeras 10 filas que corresponden a las caracteristicas de 10 flores. 
print("y:\n", y[:10])   # muestra las primeras 10 clases para cada una de las instancias de X

Podemos saber los nombres de los *features* (columnas) usando el campo **feature_names**:

In [ ]:
print(iris.feature_names)

Para saber cuáles son las clases:

In [ ]:
print(iris_df.target)  # mostramos todas las clases de X 

**0 Corresponde a setosa; 
1 Corresponde a versicolor;
2 Corresponde a virginica**


Podemos verificar esto haciendo.

In [ ]:
print(iris.target_names)   

In [ ]:
type(iris.data)

En resumen, lo que tenemos es la descripción de una flor (medidas) en una fila, acompañada con la clase. Por ejemplo, para la segunda fila, tenemos:

**4.9,  3. ,  1.4,  0.2**  y la clase asociada es **0 (setosa)**

In [ ]:
print(iris_df.iloc[0])

Podemos obtener una descripción más completa usando:

In [ ]:
print(iris.DESCR)

Lo que haremos a continuación será entrenar un clasificador y predecir con nuevos datos. 


## Nuestro primer clasificador

Entrenaremos nuestro primer clasificador con *Iris dataset*. 
Para esto, usaremos un árbol de decisión.

In [ ]:
from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier()

clf.fit(X, y)   ## Entrenar usando X (features), y (clase)

In [ ]:
# from sklearn import svm 
# clf = svm.SVC()
# clf.fit(X, y)   ## Entrenar usando X (features), y (clase)

Con el método **fit** entrenamos el clasificador con los datos de <b>X</b> y la clase asociada a cada uno, **y**. Para ver qué tan bien fue el entrenamiento, podemos evaluar el clasificador ejecutándolo sobre instancias y verificando que la clase sea la correcta. 

Ya hemos entrenado nuestro clasificador empleando **fit**. Luego para predecir, usamos **predict**.

In [ ]:
y_pred = clf.predict(X)   ## predecir 'y' usando la matriz 'X'
print(y_pred)

Por ejemplo, si ejecutáramos ```clf.predict([[5.1, 3.5, 1.4, 0.2]])```, le estamos pasando al clasificador un dato con valores **[5.1, 3.5, 1.4, 0.2]**. Al ejecutar **predict**, éste nos retornará un arreglo con el valor <b>0</b>, indicando que esos datos fueron clasificados como la clase **0 (setosa)**.

En **scikit-learn** existe el método **accuracy_score** que computa el Accuracy de la clasificación:

In [ ]:
from sklearn.metrics import accuracy_score

print("Accuracy:", accuracy_score(y, y_pred))

Como se puede apreciar, tuvimos un accuracy del 100% con el clasificador (1.0). Sin embargo, **hicimos algo incorrecto**: evaluamos el clasificador con los mismos datos con los cuales lo entrenamos! 

Al hacer esto, lo que terminamos haciendo fue *aprender de los datos de entrada* y evaluamos (o testeamos) usando los mismos datos. Esto también se denomina **overfitting**. 

También podríamos ver otras métricas como precision, recall y f-score. 

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y, y_pred))

Desde luego, la clasificación es 'perfecta'. Para tener un resultado más realista de la clasificación vamos a dividir el dataset en dos: **training set** y **test set**.

El **training_set** nos permite aprender de ejemplos y ajustar el clasificador de acuerdo a éstos. 
El **test_set** nos permitirá comprender qué tan bien **generalizamos** con nuevos datos. 

 En **scikit-learn** existe un método llamado **train_test_split**, que nos permite hacer esta separación de manera aleatoria y estratificada (es decir, manteniendo la proporción de clases entre las instancias de cada set):

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.33, random_state=37,
                                                    stratify=y)

El método retorna cuatro listas, los correspondientes a train y a test. Es decir, para el entrenamiento se usará **X_train** que tiene los features de entrenamiento e **y_train** que son sus respectivas clases. Así mismo, para probar con nuevos datos (testing), usaremos **X_test** como los features de entrada e **y_test** como las clases respectivas. 

* El parámetro **test_size** nos permite regular la fracción de los datos que irán a test; en este caso 33% del dataset completo. 

* El parámetro **random_state** nos permite fijar la semilla para tener resultados consistentes (genera la misma partición de datos). Si ejecutamos el método varias veces con la misma semilla, nos mostrará los mismos resultados siempre. 

* El parámetro **stratify** recibe un arreglo con la distribución de clases, y el método intenta mantener esa misma distribución en las listas resultantes. Si no hicieramos esto, podríamos, por ejemplo, tener muchos datos de una clase en el set de entrenamiento. 

Ahora, al fin, podemos probar nuestro clasificador:

In [ ]:
clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)    ## Entrenamos con features X_train y clases y_train

y_pred = clf.predict(X_test)   ## Predecimos con nuevos datos (los de test X_test)

print("Accuracy en test set:", accuracy_score(y_test, y_pred))   ## Evaluamos la predicción comparandbo y_test con y_pred

### Matriz de confusión

In [ ]:
## EJECUTAR ESTE BLOQUE

from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels
import seaborn as sns; sns.set(style="ticks", color_codes=True)
import matplotlib.pyplot as plt

# Compute confusion matrix. By default is not normalized (normalize=None)
# If necessary change 'y_pred' according to your variable name
cm = confusion_matrix(y_test, y_pred) 

# Only use the labels that appear in the data
classes = unique_labels(y_test, y_pred)

df = pd.DataFrame(cm, index = classes, columns = classes)

g = sns.heatmap(df, annot=True, cmap="Blues")
g.set_yticklabels(g.get_yticklabels(), rotation = 0)

plt.title('Confusion matrix \n')
plt.xlabel('Predicted label')
plt.ylabel('True label')

plt.autoscale()

# Trabajando con datos desbalanceados

Cargaremos el siguiente dataset que tiene datos desbalanceados respecto a la clase objetivo a predecir.

In [ ]:
unbalanced_path = 'https://users.dcc.uchile.cl/~hsarmien/mineria/datasets/unbalanced.csv'
data = pd.read_csv(unbalanced_path)  # abrimos el archivo csv y lo cargamos en data
data.head()

In [ ]:
print("Distribucion de clases original")
data['Class'].value_counts()

Antes de hacer algo para tratar el desbalance entre las clases primero debemos dividir en train-test.

In [ ]:
data_train, data_test, ytrain, ytest = train_test_split(data, data['Class'], test_size=0.3, stratify=data['Class'])
# proporción de clases en el train después de dividir en train-test
ytrain.value_counts()

## Clasificando con datos desbalanceados utilizando técnicas de balanceo de clases

In [ ]:
import numpy as np

print("Distribución de clases usando (over/sub) sampling")
print()

data_train = data_train.reset_index(drop=True)

# oversampling sobre la clase 1
idx = np.random.choice(data_train[data_train['Class'] == 1].index, size=69)
data_oversampled = pd.concat([data_train, data_train.iloc[idx]])
print("Data oversampled on class '1'")
print(data_oversampled['Class'].value_counts())
print()


# subsampling sobre la clase 0
idx = np.random.choice(data_train.loc[data_train.Class == 0].index, size=69, replace=False)
data_subsampled = data_train.drop(data_train.iloc[idx].index)
print("Data subsampled on class '0'")
print(data_subsampled['Class'].value_counts())

In [ ]:
data_test.columns[:-1]

In [ ]:
## ejecutar este código para preparar los datos
from sklearn.metrics import classification_report

# Preparando los data frames para ser compatibles con sklearn

# datos test (mismo para todos los conjuntos de entrenamiento)
X_test = data_test[data_test.columns[:-1]] # todo hasta la penultima columna
y_test = data_test[data_test.columns[-1]]  # la última columna

# datos entrenamiento "originales"
X_orig = data_train[data_train.columns[:-1]] 
y_orig = data_train[data_train.columns[-1]] 

# datos entrenamiento "oversampleados" 
X_over = data_oversampled[data_train.columns[:-1]]
y_over = data_oversampled[data_train.columns[-1]]

# datos entrenamiento "subsampleados"
X_subs = data_subsampled[data_train.columns[:-1]]
y_subs = data_subsampled[data_train.columns[-1]]

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

## Pasos:
##  - instanciar el clasificador con DecisionTreeClassifier()
##  - entrenar con fit()
##  - hacer las predicciones
##  - mostrar precision, recall y f1-score con classification report.

print("ORIGINAL::::::::::")
clf_orig = DecisionTreeClassifier()
clf_orig.fit(X_orig, y_orig)
pred_orig = clf_orig.predict(X_test)
print(classification_report(y_test, pred_orig))

###########PAUTA#############

print("OVERSAMPLING::::::::::")
clf_over = DecisionTreeClassifier()
clf_over.fit(X_over, y_over)
pred_over = clf_over.predict(X_test)
print(classification_report(y_test, pred_over))

print("SUBSAMPLING::::::::::")
clf_subs = DecisionTreeClassifier()
clf_subs.fit(X_subs, y_subs)
pred_subs = clf_subs.predict(X_test)
print(classification_report(y_test, pred_subs))

## Evaluando múltiples algoritmos de clasificación

In [ ]:
## run_classifier recibe un clasificador y un dataset (X, y)
## y opcionalmente la cantidad de resultados que se quiere obtener del clasificador

from sklearn.metrics import f1_score, recall_score, precision_score
from sklearn.model_selection import train_test_split
import numpy as np

def run_classifier(clf, X, y, num_tests=100):
    metrics = {'f1-score': [], 'precision': [], 'recall': []}
    
    for _ in range(num_tests):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.30)

        clf.fit(X_train, y_train)
        predictions = clf.predict(X_test)
        
        metrics['y_pred'] = predictions
        metrics['f1-score'].append(f1_score(y_test, predictions)) 
        metrics['recall'].append(recall_score(y_test, predictions))
        metrics['precision'].append(precision_score(y_test, predictions))
    return metrics

In [ ]:
data[data.columns[-1]]

In [ ]:
from sklearn.dummy import DummyClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB  # naive bayes
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC  # support vector machine classifier

#bc = load_breast_cancer()    # dataset cancer de mamas
X = data[data.columns[:-1]]
y = data[data.columns[-1]]

c0 = ("Base Dummy", DummyClassifier(strategy='stratified'))
c1 = ("Decision Tree", DecisionTreeClassifier(max_depth=5))
c2 = ("Gaussian Naive Bayes", GaussianNB())
c3 = ("KNN", KNeighborsClassifier(n_neighbors=10))
c4 = ("Support Vector Machines", SVC())

classifiers = [c0, c1, c2, c3, c4]

results = {}
for name, clf in classifiers:
    metrics = run_classifier(clf, X, y)   # hay que implementarla en el bloque anterior.
    results[name] = metrics
    print("----------------")
    print("Resultados para clasificador: ", name) 
    print("Precision promedio:", np.array(metrics['precision']).mean())
    print("Recall promedio:", np.array(metrics['recall']).mean())
    print("F1-score promedio:", np.array(metrics['f1-score']).mean())
    print("----------------\n\n")  

# Clasificación de texto

In [ ]:
import os
cwd = os.getcwd()
df = pd.read_csv(os.path.join(cwd, "tfidf_tweets.csv"),
                         sep = ";", 
                         encoding = "utf-8",
                         quotechar = "\"") 

In [ ]:
df.shape

In [ ]:
X = np.array(df[df.columns[:-1]])
y = df[df.columns[-1]]

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.33, random_state=37,
                                                    stratify=y)

In [ ]:
clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)    ## Entrenamos con features X_train y clases y_train

y_pred = clf.predict(X_test)   ## Predecimos con nuevos datos (los de test X_test)

print("Accuracy en test set:", accuracy_score(y_test, y_pred))   ## Evaluamos la predicción comparandbo y_test con y_pred

In [ ]:
print(classification_report(y_test, y_pred))

### Matriz de confusión

In [ ]:
## EJECUTAR ESTE BLOQUE

from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels
import seaborn as sns; sns.set(style="ticks", color_codes=True)
import matplotlib.pyplot as plt

# Compute confusion matrix. By default is not normalized (normalize=None)
# If necessary change 'y_pred' according to your variable name
cm = confusion_matrix(y_test, y_pred) 

# Only use the labels that appear in the data
classes = unique_labels(y_test, y_pred)

df = pd.DataFrame(cm, index = classes, columns = classes)

g = sns.heatmap(df, annot=True, cmap="Blues")
g.set_yticklabels(g.get_yticklabels(), rotation = 0)

plt.title('Confusion matrix \n')
plt.xlabel('Predicted label')
plt.ylabel('True label')

plt.autoscale()

# Referencias, lectura adicional

Documentación de Pandas: https://pandas.pydata.org/pandas-docs/stable/user_guide/index.html 

Fusionar, unir, concatenar y comparar dataframes: https://pandas.pydata.org/docs/user_guide/merging.html

Documentación de scikit-learn: https://scikit-learn.org/stable/user_guide.html

Tutoriales scikit-learn: https://scikit-learn.org/stable/tutorial/index.html, https://scikit-learn.org/stable/auto_examples/index.html

Galería Seaborn: https://seaborn.pydata.org/examples/index.html

Galería matplotlib: https://matplotlib.org/stable/plot_types/index.html

